In [1]:
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout , MaxPooling2D, BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping, TensorBoard
from matplotlib import pyplot as plt
import os, pandas as pd, cv2

In [2]:
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder, LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
import numpy as np
from numpy import expand_dims
import shutil

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip "drive/My Drive/affectnethq.zip" -d "/content/affectnet"

In [36]:
df = pd.read_csv('affectnet/labels.csv')
df['label'] = df['label'].str.lower()

df['pth'] = 'affectnet/' + df['pth'].astype(str)
labels = df['label'].unique()
lbl_cnt = df['label'].value_counts()

<h2> Dataset processing using file commands </h2>

In [62]:
train_split = 80
valid_split = 20
curr_lbl_cnt = {}
emotion_set = {'anger', 'fear', 'happy', 'sad', 'surprise', 'neutral'}

In [63]:
shutil.rmtree('proper_affectnet')

In [64]:
path = '/content/proper_affectnet'

os.mkdir(path)
os.mkdir(path+'/train')
os.mkdir(path+'/valid')
os.mkdir(path+'/test')

In [ ]:
for emotion in os.listdir('affectnet'):

    if emotion not in emotion_set:
        continue

    tot_count = 3400
    curr_count = 0

    for file_name in os.listdir('affectnet/'+emotion):

        ori_path = 'affectnet/'+emotion+'/'+file_name

        if(curr_count < int((tot_count*train_split)/100)):
        
            if not os.path.exists(path+'/train/'+emotion):
                os.mkdir(path+'/train/'+emotion)

            shutil.copy(ori_path, path+'/train/'+emotion+'/')
        
        elif(curr_count < int((tot_count*(train_split+valid_split))/100)):

            if not os.path.exists(path+'/valid/'+emotion):
                os.mkdir(path+'/valid/'+emotion)

            shutil.copy(ori_path, path+'/valid/'+emotion+'/')
        
        elif(curr_count < tot_count):

            if not os.path.exists(path+'/test/'+emotion):
                os.mkdir(path+'/test/'+emotion)

            shutil.copy(ori_path, path+'/test/'+emotion+'/')
        
        else: break

        curr_count += 1

        print('pasted img no '+str(curr_count) +' of emotion '+emotion)


In [66]:
for directory in os.listdir('affectnet'):

    if 'labels' not in directory:
        print(directory, len(os.listdir('affectnet/'+directory)))

fear 3622
neutral 5132
surprise 4296
contempt 3179
sad 3430
disgust 2660
happy 5045
anger 3638


In [67]:
train_dir = path+'/train'
test_dir = path+'/test'
val_dir = path+'/valid'

In [68]:
train_datagen = ImageDataGenerator(rescale=1.0/255)
train_generator = train_datagen.flow_from_directory(
    directory=train_dir,
    target_size=(224,224),
    class_mode='categorical',
    batch_size=32)

val_datagen = ImageDataGenerator(rescale=1.0/255)
val_generator = val_datagen.flow_from_directory(
    directory=val_dir,
    target_size=(224,224),
    class_mode='categorical',
    batch_size=32)

# test_datagen = ImageDataGenerator(rescale=1.0/255)
# test_generator = test_datagen.flow_from_directory(
#     directory=test_dir,
#     target_size=(224,224),
#     class_mode='categorical',
#     batch_size=64)

Found 16320 images belonging to 6 classes.
Found 4080 images belonging to 6 classes.


<h2> Dataset processing using dataframe</h2>

In [37]:
train_df, test_df = train_test_split(df , test_size = 0.2 , stratify = df['label'] , random_state = 0)

In [54]:
train_df['label'].value_counts()

surprise    3911
happy       3505
anger       3328
disgust     3021
fear        3002
contempt    2870
sad         2682
neutral     2482
Name: label, dtype: int64

In [ ]:
train_datagen = ImageDataGenerator(rescale=1.0/255)
train_generator = train_datagen.flow_from_dataframe(
    dataframe = train_df,
    x_col = "pth",
    y_col = "label",
    target_size=(224,224),
    class_mode='categorical',
    batch_size=32)

val_datagen = ImageDataGenerator(rescale=1.0/255)
val_generator = val_datagen.flow_from_dataframe(
    dataframe = train_df,
    x_col = "pth",
    y_col = "label",
    target_size=(224,224),
    class_mode='categorical',
    batch_size=32)

<h2> Creating the Model</h2> 

In [69]:
pre_model = VGG19(weights='imagenet', 
                  input_shape=(224,224,3), 
                  include_top=False
                  )

In [70]:
for layer in pre_model.layers:
    layer.trainable = False

In [71]:
for layer in pre_model.layers:
    print(layer, layer.trainable)

<keras.engine.input_layer.InputLayer object at 0x7fee48159f70> False
<keras.layers.convolutional.conv2d.Conv2D object at 0x7fee48263b80> False
<keras.layers.convolutional.conv2d.Conv2D object at 0x7fee4b1b83a0> False
<keras.layers.pooling.max_pooling2d.MaxPooling2D object at 0x7fee482bf190> False
<keras.layers.convolutional.conv2d.Conv2D object at 0x7fee4b510be0> False
<keras.layers.convolutional.conv2d.Conv2D object at 0x7fee4b56ec10> False
<keras.layers.pooling.max_pooling2d.MaxPooling2D object at 0x7fee482bf250> False
<keras.layers.convolutional.conv2d.Conv2D object at 0x7fee48162f40> False
<keras.layers.convolutional.conv2d.Conv2D object at 0x7fee4813fa30> False
<keras.layers.convolutional.conv2d.Conv2D object at 0x7fef51d328b0> False
<keras.layers.convolutional.conv2d.Conv2D object at 0x7fef51d38640> False
<keras.layers.pooling.max_pooling2d.MaxPooling2D object at 0x7fee48162a90> False
<keras.layers.convolutional.conv2d.Conv2D object at 0x7fef51d32e80> False
<keras.layers.convolut

In [79]:
model = Sequential([
    pre_model,
    Dropout(0.3),
    Flatten(),
    Dense(8192, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(1024, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.4),
    Dense(6 , activation='softmax')])

model.compile(optimizer = "adam" , loss = 'categorical_crossentropy' , metrics = ['accuracy'])
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 7, 7, 512)         20024384  
                                                                 
 dropout_1 (Dropout)         (None, 7, 7, 512)         0         
                                                                 
 flatten_3 (Flatten)         (None, 25088)             0         
                                                                 
 batch_normalization_1 (Batc  (None, 25088)            100352    
 hNormalization)                                                 
                                                                 
 dense_10 (Dense)            (None, 512)               12845568  
                                                                 
 batch_normalization_2 (Batc  (None, 512)              2048      
 hNormalization)                                      

In [80]:
model_name = 'emotion_vgg19_custom.h5'
checkpointer = ModelCheckpoint(model_name, verbose=1, save_best_only=True)
early_stopper = EarlyStopping(patience = 11, monitor='val_loss')

callbacks = [checkpointer, early_stopper]

In [81]:
epochs = 50

In [82]:
history = model.fit(
    x=train_generator,
    steps_per_epoch=len(train_generator)//32,
    epochs=epochs,
    callbacks = callbacks,
    validation_data=val_generator,
    validation_steps=len(val_generator)//32,
    verbose = 1)

Epoch 1/50
15/15 [==============================] - ETA: 0s - loss: 3.0456 - accuracy: 0.3292
Epoch 1: val_loss improved from inf to 2.99464, saving model to emotion_vgg19_custom.h5
15/15 [==============================] - 14s 865ms/step - loss: 3.0456 - accuracy: 0.3292 - val_loss: 2.9946 - val_accuracy: 0.3047
Epoch 2/50
15/15 [==============================] - ETA: 0s - loss: 2.6461 - accuracy: 0.3729
Epoch 2: val_loss improved from 2.99464 to 1.58053, saving model to emotion_vgg19_custom.h5
15/15 [==============================] - 12s 839ms/step - loss: 2.6461 - accuracy: 0.3729 - val_loss: 1.5805 - val_accuracy: 0.4922
Epoch 3/50
15/15 [==============================] - ETA: 0s - loss: 2.1258 - accuracy: 0.4396
Epoch 3: val_loss improved from 1.58053 to 1.47375, saving model to emotion_vgg19_custom.h5
15/15 [==============================] - 13s 875ms/step - loss: 2.1258 - accuracy: 0.4396 - val_loss: 1.4737 - val_accuracy: 0.4609
Epoch 4/50
15/15 [==============================] 